# Cloudflare

[![Open in Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/adamelliotfields/ml/blob/main/notebooks/cloudflare.ipynb)
[![Render nbviewer](https://img.shields.io/badge/render-nbviewer-f37726)](https://nbviewer.org/github/adamelliotfields/ml/blob/main/notebooks/cloudflare.ipynb)

This is my opinionated setup for networking in Colab. It assumes you already have an existing [Cloudflare Tunnel](https://www.cloudflare.com/products/tunnel). If not, read my [gist](https://gist.github.com/adamelliotfields/9e3610eecef19be5d38b87a2caec4912) on how to create a tunnel with a custom domain name.

**Why?** Colab doesn't expose your VM to the internet. Cloudflare Tunnel creates a secure connection between your device and Cloudflare's network (the internet).

My approach follows 3 rules:
  1. Use the GUI button to mount Google Drive. Using `drive.mount()` in Python requires authenticating every time.
  2. Store Cloudflare configuration in Drive. The only thing that ever changes is the port number.
  3. Manage the tunnel with SysV. Avoid shell scripts and separate terminals.

It requires 2 text files: `config.yml` and `${UUID}.json`. The UUID is unique to your tunnel. The JSON file is generated after running `cloudflared tunnel create` for the first time.

You create `config.yml` manually and it looks like this:

```yaml
tunnel: $UUID
url: http://localhost:8000
credentials-file: /content/drive/MyDrive/cloudflared/$UUID.json
```

Both files go in `/content/drive/MyDrive/cloudflared` with `config.yml` being the source-of-truth. You only have to do this once.

In [ ]:
# @markdown ## Config
PORT = 8000  # @param {type: "integer"}

In [ ]:
# @title Deps
import os
import yaml
import shutil
import requests
import subprocess

In [ ]:
# @title Install cloudflared
if not shutil.which("cloudflared"):
    # colab is ubuntu amd64
    file = "cloudflared-linux-amd64.deb"
    r = requests.get(f"https://github.com/cloudflare/cloudflared/releases/latest/download/{file}")
    with open(f"/tmp/{file}", "wb") as f:
        f.write(r.content)
    subprocess.run(["dpkg", "-i", f"/tmp/{file}"])

# cloudflared copies the remote config to /etc/cloudflared when installing the service
# if the local config exists, install exits with a non-zero
if not os.path.exists("/etc/cloudflared/config.yml"):
    # set port in config
    cloudflared_home = "/content/drive/MyDrive/cloudflared"
    with open(f"{cloudflared_home}/config.yml", "r") as f:
        config = yaml.safe_load(f)
    with open(f"{cloudflared_home}/config.yml", "w") as f:
        config["url"] = f"http://localhost:{PORT}" if PORT else config["url"]
        yaml.safe_dump(config, f)
    # install service
    subprocess.run(
        ["cloudflared", "--config", f"{cloudflared_home}/config.yml", "service", "install"]
    )

# print tunnel status
process = subprocess.run(["service", "cloudflared", "status"], capture_output=True, text=True)
print(f"Tunnel: {process.stdout}")  # "Running" or "Stopped"

In [ ]:
# @title Run web app
# (for demo only)
with open("/tmp/index.html", "w") as f:
    f.write("<h1>Hello World</h1>")

# ensure the tunnel is up: https://one.dash.cloudflare.com
subprocess.run(["service", "cloudflared", "restart"])

# run the server, pipe output back to main process and decode (text=True)
process = subprocess.Popen(
    ["python", "-m", "http.server", "--directory", "/tmp", str(PORT if PORT else 8000)],
    stderr=subprocess.STDOUT,
    stdout=subprocess.PIPE,
    text=True,
)

# stopping the web app also stops the tunnel service even if handled gracefully
try:
    for line in iter(process.stdout.readline, ""):
        print(line.strip())
except KeyboardInterrupt:
    pass
finally:
    process.terminate()
    process.wait()